# Grove Motor Driver v1.3  
http://wiki.seeedstudio.com/Grove-I2C_Motor_Driver_V1.3/

In [1]:
import time

from meerkat import motor

#### Stop Command for Debugging  
This cell is useful when prototyping.  If a command throws an error before setting both motor speeds to 0, the board will remain on and possibly overheat.

In [2]:
m = motor.GroveMotor(bus_n=1)
m.stop()
print("Stop Done")

Stop Done


#### I2C Bus Address
Initialize the motor controller class.  Default I2C address is `0x0F` which the four switches are set to `1111` - another option is to use Python's binary representation `0b00001111`

In [3]:
hex(0b00001111)

'0xf'

#### Initialize Driver Class with Default I2C Address

In [4]:
m = motor.GroveMotor(bus_n=1)

#### Motor Metadata

In [5]:
m.writer.device

{'name': 'Grove Motor Driver',
 'version_hw': '1.3',
 'version_sw': '1.0',
 'accuracy': None,
 'precision': '1 microstep',
 'bus': '<meerkat.i2c_pi.WrapI2C object at 0x72af9cd0>',
 'state': None,
 'active': None,
 'error': None,
 'dtype': None,
 'description': 'I2C DC and stepper motor controller',
 'urls': 'http://wiki.seeedstudio.com/Grove-I2C_Motor_Driver_V1.3/',
 'manufacturer': 'Seeed Studio',
 'calibration_date': None}

#### Motor Status Header

In [6]:
m.writer.header

['description',
 'freq',
 'm1_speed',
 'm2_speed',
 'm_direction',
 'mode',
 'phase',
 'steps']

#### Motor Status

In [7]:
m.get("test_1_init")

['test_1_init', 31372, 0, 0, '', 'dc', 2, 0]

To be clear, the headers names aligned with the data above are:  

| description | freq | m1_speed | m2_speed | m_direction | mode | phase | steps |
| ----------- | ---- | -------- | -------- | ----------- | ---- | ----- | ----- |
| test_1_init | 31372 | 0 | 0 |  | dc | 2 | 0 |

## Stepper Motor  

#### Micro Steps

In [8]:
m.step_micro(1001, delay=0.0001, verbose=False)

time.sleep(1)

m.step_micro(-1001, delay=0.0001, verbose=False)

In [9]:
m.get("test_2_microsteps")

['test_2_microsteps', 31372, 0, 0, 'step_ccw', 'micro_step', 2, -1001]

#### Full Steps

In [10]:
m.step_full(100, delay=0.0001, verbose=False)

time.sleep(1)

m.step_full(-100, delay=0.0001, verbose=False)

In [11]:
m.get("test_3_full_steps")

['test_3_full_steps', 31372, 0, 0, 'step_ccw', 'full_step', 2, -100]

## DC Motor  
For this portion, disconnect the stepper motor from pinout J3 and connect a DC motor to J1.  

Note: The Grove Arduino driver uses -100 to 100 for accepted motor speed values, however their driver converts it to -255 to 255 because that is the board's accepted input values.  -255 to 255 is used here so that finer step control is possible.  Bench test speeds needed for a motor, if 100 is used with this Python driver, some motors may not start!

### LED Colors, Directions Codes and Descriptions

| M1 | M2 | Code | Description |
| -- | -- | ---- | ----------- |
| <span style='color:green'>green</span> | <span style='color:green'>green</span> | cw         | both clockwise         |
| <span style='color:red'>red</span>     | <span style='color:red'>red</span>     | ccw        | both counter clockwise |
| <span style='color:green'>green</span> | <span style='color:green'>green</span> | m1m2cw     | both clockwise         |
| <span style='color:red'>red</span>     | <span style='color:red'>red</span>     | m1m2cc     | both counter clockwise |
| <span style='color:green'>green</span> | <span style='color:red'>red</span>     | m1cw_m2ccw | M1 clockwise, M2 counter clockwise |
| <span style='color:red'>red</span>     | <span style='color:green'>green</span> | m1ccw_m2cw | M1 counter clockwise, M2 clockwise |

In [12]:
# ensure stop condition before switching to dc motor control
m.stop()

In [13]:
m.set_mode(mode_type="dc")
m.set_frequency(f_Hz=31372)

#### DC Motor Clockwise

In [14]:
m.set_direction('cw')
m.set_speed(motor_id=1, motor_speed=255)

time.sleep(4)

m.stop(motor_id=1)

In [15]:
m.get("test_4_dc_clockwise")

['test_4_dc_clockwise', 1, 0, 0, 10, 'dc', 2, -100]

#### DC Motor Counter Clockwise

In [16]:
m.set_frequency(f_Hz=31372)
m.set_direction('ccw')
m.set_speed(motor_id=1, motor_speed=255)
time.sleep(4)
m.stop(motor_id=1)

In [17]:
m.get("test_5_dc_counter_clockwise")

['test_5_dc_counter_clockwise', 1, 0, 0, 5, 'dc', 2, -100]